### 1- Need to generate Adult Data and pass it in self.data
<ul>
   <li>Need ask best params from Rosin</li>
    <li>
</ul>
### 2- in Dataloader class, self.labels can be directly the original as 2d tensors


### Imports

In [1]:
# # Imports
# from Modules import setup as s
# from Modules import utils as u
# from Modules import plots as pl
# from Modules import models as m
# from Modules import Results as r
import datasets as d
# from Modules import metrics_ as me
# from Modules import customLosses as cl
# from Modules import analysisTools as at
# from Modules import optimizers2 as optim
# from Parameters import parameters as p

# from Stats import Plots as Pl
import tqdm
import torch
import warnings
import importlib
import random
import torchvision
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from typing import List
from torch.utils.data import *
import matplotlib.pyplot as plt
from torch.utils import data as td
from joblib import Parallel, delayed

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CPU_DEVICE = torch.device("cpu")
GET_VALUE = lambda x: x.to(CPU_DEVICE).data.numpy().reshape(-1)[0]



In [2]:
# Data import & Pre-processing

def dummy_encode(df: pd.core.frame.DataFrame, cat_cols : List[str] ) -> pd.core.frame.DataFrame:
    '''
        Will first look for categorical variable (strings) in each columns' first value, 
        then encode the categorical columns in one-hot using pandas dummy encoding
        
        :param df: Dataframe to work with
        :param cat_cols: list of columns that needs to be changed 
        
        :return same dataframe, encoded if required
        
    '''
    if cat_cols == 0:
        
        return df
    else:
        return(pd.get_dummies(df2, columns=cat_cols))
    

def find_cat(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    cat_cols = []
    for col_name, col_data in df.iteritems():
        if type(col_data.values[0]) == str:
            cat_cols.append(col_name)

    return cat_cols
            
class My_dataLoader:
    def __init__(self, batch_size : int, path :str, n_train :int, test_batch:int = None, label_col_name:str):
        '''
            Creates train and test loaders from local files, to be easily used by torch.nn
            
            :batch_size: int for size of training batches
            :path: path to csv where data is. 2d file, where last value of each cols
                    is the label
            :n_train: int for the size of training set (assigned randomly)
            :test_batch: size of batches at test time. If none, will be same 
                    as training
            :label_col_name name of columns that contains labels
        '''
        self.batch_size = batch_size
        self.path = path
        self.n_test = n_test

        batch_size_eval = 128
        
        df = pd.read_csv(path)
        
        
        a = df.values
        self.data = torch.tensor(a[:,:-1]) # where data is 2d [D_train_size x features]
        self.labels = torch.tensor(a[:,-1])
        
        self.local_dataset = torch.utils.data.TensorDataset(self.data, self.labels)
                     
        #custom_transform = transforms.Normalize((mean_mean,), (std_mean,)) 
        
        
        indices = list(range(len(self.local_dataset)))
        random.shuffle(indices)
        
        # Split dataset into train and Test sets
        self.train_loader = torch.utils.data.DataLoader(
            self.local_dataset,
            batch_size=self.batch_size,
            sampler=SubsetRandomSampler(indices[:n_test]),
            num_workers=1,
            pin_memory=False
        )

        self.test_loader = DataLoader(
            self.local_dataset,
            batch_size=batch_size_eval,
            sampler=SubsetRandomSampler(indices[n_test:]),
            num_workers=1,
            pin_memory=False
        )

        
# Create the object with both loader and loss functions
batchSize = 32

# dataloader = My_dataLoader(batchSize, path, n_test)


In [5]:
# Import Data
import_path = "../GeneralDatasets/Csv/Adult_NoCat"
df2 = pd.read_csv(import_path+".csv")
n_test = int(len(df2.iloc[:,0])*0.85) # keep 15% for test set

# Encode Categorical Vars
cat_list = find_cat(df2)


if len(cat_list) > 0:
    print(f"Categorical variable found. Running Pandas dummy_variable encoding on: \n [{cat_list}] \n")
    df2 = dummy_encode(df2, cat_list)
    print(f"Saving output dataset under {import_path}_NoCat.csv \n")
    df2.to_csv(f"{import_path}_NoCat.csv", index=False)
# Load data
    data_loader = My_dataLoader(batchSize, f"{import_path}_NoCat.csv", n_test, "income")
else:
    data_loader = My_dataLoader(batchSize, import_path+'.csv', n_test, "income")


In [72]:
# group_attributes = ["label"]  # Protected or sensitive attribute

# # TODO: Change what is put as labels, if I want to use pytorch loss. Might have to deal with L1 calculation manually
# d_ = d.TorchGeneralDataset("../test.csv", target_feature=group_attributes[0],
#                                        xTensor=torch.FloatTensor, yTensor=torch.LongTensor,
#                                        transform=None,
#                      )

# l_d = td.DataLoader(d_, batch_size=d_i.df.shape[0], shuffle=False, num_workers=p.Num_workers)

In [10]:
# Box

class autoencoder(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, out_dim), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [13]:
data_loader.labels

tensor([0, 0, 0,  ..., 0, 0, 0])

### Training

In [12]:
len(df2.iloc[0,:])

104

In [ ]:
wd = None
learning_rate = 0.001
num_epochs = 2
#######
in_dim = len(df2.iloc[0,:])
out_dim = len(df2.iloc[0,:])
#######

model = autoencoder(in_dim, out_dim).cuda()
train_loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wd)


for epoch in range(num_epochs):
    for data in dataloader.train_data:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # Fprop
        output = model(img)
        loss = train_loss(output, img)
        # bprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        NEED TEST SET
        
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')